<a href="https://colab.research.google.com/github/silverstar0727/pose-estimation/blob/main/continuous_training_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# 해당 셀을 실행한 후에 반드시 "런타임 다시시작"을 해주세요
!pip install -q kfp
!pip3 install --user kfp google-cloud-aiplatform matplotlib --upgrade -q
# 추가
!pip3 install --user google-cloud-aiplatform --upgrade -q
!pip3 install --user kfp google-cloud-pipeline-components --upgrade -q

     |████████████████████████████████| 1.3 MB 4.8 MB/s 
     |████████████████████████████████| 10.3 MB 19.0 MB/s 
     |████████████████████████████████| 42 kB 1.7 MB/s 
     |████████████████████████████████| 196 kB 60.9 MB/s 
     |████████████████████████████████| 4.3 MB 47.4 MB/s 
     |████████████████████████████████| 196 kB 58.9 MB/s 
     |████████████████████████████████| 196 kB 53.2 MB/s 
     |████████████████████████████████| 195 kB 57.7 MB/s 
     |████████████████████████████████| 194 kB 57.7 MB/s 
     |████████████████████████████████| 193 kB 58.6 MB/s 
     |████████████████████████████████| 189 kB 62.9 MB/s 
     |████████████████████████████████| 188 kB 55.3 MB/s 
     |████████████████████████████████| 188 kB 52.8 MB/s 
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the sou

In [2]:
import kfp
from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output,
                        OutputPath, component, ClassificationMetrics, Metrics)
from kfp.v2.google.client import AIPlatformClient
from kfp.v2.google import experimental
from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip

In [12]:
from google.colab import auth as google_auth

google_auth.authenticate_user() # 사용할 gcp 계정으로 연결해주세요

In [13]:
from datetime import datetime

PROJECT_ID = 'natural-expanse-319203'
REGION = "us-central1"

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET_NAME = "gs://mediapipe-pipeline"

USER = "JeongMin-Do"
PIPELINE_ROOT = "{}/pipeline_root/{}".format(BUCKET_NAME, USER)

In [14]:
@component(base_image="silverstar456/mediapipe:landmarks")
def get_landmarks(result_csv: OutputPath("result_csv")):
    import csv
    import cv2
    import numpy as np
    import os
    import sys
    import tqdm
    import pandas as pd

    from mediapipe.python.solutions import drawing_utils as mp_drawing
    from mediapipe.python.solutions import pose as mp_pose

    import wget 
    import zipfile

    # GCS에서 이미지 파일 다운로드 
    wget.download("https://storage.googleapis.com/mediapipe-pipeline/study.zip")

    # 압축해제
    fantasy_zip = zipfile.ZipFile('study.zip')
    fantasy_zip.extractall('.')
    fantasy_zip.close()

    def landmarks(input_frame):
        input_frame = cv2.cvtColor(input_frame, cv2.COLOR_BGR2RGB)

        # Initialize fresh pose tracker and run it.
        with mp_pose.Pose(upper_body_only=True) as pose_tracker:
            result = pose_tracker.process(image=input_frame)
            pose_landmarks = result.pose_landmarks
        
        # Save landmarks.
        if pose_landmarks is not None:
            # Check the number of landmarks and take pose landmarks.
            assert len(pose_landmarks.landmark) == 25, 'Unexpected number of predicted pose landmarks: {}'.format(len(pose_landmarks.landmark))
            pose_landmarks = [[lmk.x, lmk.y, lmk.z] for lmk in pose_landmarks.landmark]

            # Map pose landmarks from [0, 1] range to absolute coordinates to get
            # correct aspect ratio.
            frame_height, frame_width = input_frame.shape[:2]
            pose_landmarks *= np.array([frame_width, frame_height, frame_width])

            # Write pose sample to CSV.
            pose_landmarks = np.around(pose_landmarks, 5).flatten().astype(np.float64).tolist()

            return pose_landmarks

    images_in_folder = 'study'
    csv_out_path = result_csv

    with open(csv_out_path, 'w') as csv_out_file:
        csv_out_writer = csv.writer(csv_out_file, delimiter=',', quoting=csv.QUOTE_MINIMAL)

        # Folder names are used as pose class names.
        pose_class_names = sorted([n for n in os.listdir(images_in_folder) if not n.startswith('.')])

        for pose_class_name in pose_class_names:
            print('Bootstrapping ', pose_class_name, file=sys.stderr)

            image_names = sorted([
                n for n in os.listdir(os.path.join(images_in_folder, pose_class_name))
                if not n.startswith('.')])
            for image_name in tqdm.tqdm(image_names, position=0):
                # Load image.
                input_frame = cv2.imread(os.path.join(images_in_folder, pose_class_name, image_name))
                pose_landmarks = landmarks(input_frame)

                try:
                    csv_out_writer.writerow([image_name, pose_class_name] + pose_landmarks)
                except:
                    pass       

In [15]:
@component(base_image="silverstar456/mediapipe:knn")
def knn(result_csv: InputPath("result_csv"), model_output: OutputPath("model")):
    import pandas as pd

    import pickle
    from joblib import dump, load
    from sklearn.neighbors import KNeighborsClassifier

    data = pd.read_csv(result_csv, header=None)
    x_train, y_train = data.iloc[:, 2:], data.iloc[:, 1]

    classifier = KNeighborsClassifier(n_neighbors = 3)

    classifier.fit(x_train, y_train)

    dump(classifier, model_output) 

    # clf = load('filename.joblib') 

In [18]:
@dsl.pipeline(
    name = "mediapipe-pipeline",
    description = "mediapipe",
    pipeline_root=PIPELINE_ROOT
)
def mediapipe():
    landmarks = get_landmarks()
    model = knn(landmarks.output)


    model_upload_op = gcc_aip.ModelUploadOp(
        project=PROJECT_ID, # 프로젝트 명
        display_name="mediapipe-pose", # 모델 이름 설정
        artifact_uri='gs://mediapipe-pipeline/pipeline_root/JeongMin-Do', # 모델이 저장된 directory
        serving_container_image_uri= "us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-cpu.2-3:latest", # serving시 사용할 컨테이너 이미지
    )
    model_upload_op.after(model)


compiler.Compiler().compile(
    pipeline_func = mediapipe, 
    package_path = "mediapipe.json"
)

api_client = AIPlatformClient(
    project_id=PROJECT_ID,
    region=REGION,
)

response = api_client.create_run_from_job_spec(
    job_spec_path="mediapipe.json"
)

/usr/local/lib/python3.7/dist-packages/kfp/v2/google/client/client.py:175: FutureWarning: AIPlatformClient will be deprecated in v1.9. Please use PipelineJob https://googleapis.dev/python/aiplatform/latest/_modules/google/cloud/aiplatform/pipeline_jobs.html in Vertex SDK. Install the SDK using "pip install google-cloud-aiplatform"
  category=FutureWarning,
